## **1. load\_obj() Function:**

*   **Purpose:** This function is responsible for parsing and loading a 3D model from an OBJ file.
    
*   **Input:** file\_path (string): The path to the OBJ file.
    
*   **Output:** A tuple containing lists of:
    
    *   vertices (list of lists of floats): Coordinates of each vertex (x, y, z).
        
    *   textures (list of lists of floats): Texture coordinates (u, v, w - w is optional).
        
    *   normals (list of lists of floats): Normal vectors for each vertex (x, y, z).
        
    *   faces (list of lists of tuples): Information about each face. Each face is a list of vertex indices (and optionally texture and normal indices). The tuple structure is (vertex\_index, texture\_index, normal\_index).
        
    *   groups (dictionary): Mapping of group names to the indices of the faces belonging to that group.
        
    *   materials (dictionary): Currently not populated in this code, intended for material information.
        
    *   mtl\_file (string or None): The name of the associated MTL (Material Template Library) file, if specified in the OBJ file.
        
*   **How it works:**
    
    *   It opens the OBJ file and reads it line by line.
        
    *   It ignores empty lines and lines starting with # (comments).
        
    *   It identifies keywords like v (vertex), vt (texture coordinate), vn (normal vector), f (face), g (group), usemtl (use material), and mtllib (material library file).
        
    *   Based on the keyword, it parses the subsequent data and appends it to the respective lists.
        
    *   **Face Parsing:** The face data is particularly important. It handles different formats of face definitions (e.g., v1, v1/vt1, v1/vt1/vn1). It also handles negative indices (which refer to vertices from the end of the list). Indices are 0-based after processing.

In [88]:
import numpy as np
from collections import defaultdict
import heapq

def load_obj(file_path):
    vertices = []
    textures = []
    normals = []
    faces = []
    groups = {}
    materials = {}
    current_group = None
    mtl_file = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split()
            keyword = parts[0]

            if keyword == 'v':
                vertices.append(list(map(float, parts[1:])))
            elif keyword == 'vt':
                textures.append(list(map(float, parts[1:])))
            elif keyword == 'vn':
                normals.append(list(map(float, parts[1:])))
            elif keyword == 'f':
                face_data = []
                for vertex_part in parts[1:]:
                    indices = vertex_part.split('/')
                    v_index = int(indices[0])
                    if v_index < 0:
                        v_index += len(vertices)
                    v_index -= 1

                    vt_index = None
                    if len(indices) > 1 and indices[1]:
                        vt_index = int(indices[1])
                        if vt_index < 0:
                            vt_index += len(textures)
                        vt_index -= 1

                    vn_index = None
                    if len(indices) > 2 and indices[2]:
                        vn_index = int(indices[2])
                        if vn_index < 0:
                            vn_index += len(normals)
                        vn_index -= 1

                    face_data.append((v_index, vt_index, vn_index))
                faces.append(face_data)
                if current_group:
                    if current_group not in groups:
                        groups[current_group] = []
                    groups[current_group].append(len(faces) - 1) # Stocker l'index de la face
            elif keyword == 'g':
                if len(parts) > 1:
                    current_group = parts[1]
            elif keyword == 'usemtl':
                if len(parts) > 1:
                    current_material = parts[1]
                    # Vous pouvez choisir de stocker l'association face -> matériau si nécessaire
            elif keyword == 'mtllib':
                if len(parts) > 1:
                    mtl_file = parts[1]
                    # Vous pouvez ajouter une fonction pour parser le fichier .mtl ici

    return vertices, textures, normals, faces, groups, materials, mtl_file

## **2. vertex\_clustering() Function:**

*   **Purpose:** This function simplifies the mesh by grouping nearby vertices into cells of a 3D grid and replacing them with a single representative vertex (the barycenter or average position).
    
*   **Inputs:**
    
    *   vertices (list of lists of floats): Original vertex coordinates.
        
    *   textures (list of lists of floats): Original texture coordinates.
        
    *   normals (list of lists of floats): Original normal vectors.
        
    *   faces (list of lists of tuples): Original face data.
        
    *   grid\_resolution (int or list of ints): Defines the resolution of the 3D grid. If an integer is provided, the grid will be uniform in all dimensions. If a list of three integers is provided, it defines the resolution for x, y, and z dimensions separately.
        
*   **Output:** A tuple containing lists of:
    
    *   new\_vertices (list of lists of floats): Simplified vertex coordinates (barycenters).
        
    *   new\_textures (list of lists of floats): Average texture coordinates for each cell (if textures exist).
        
    *   new\_normals (list of lists of floats): Average normal vectors for each cell (if normals exist).
        
    *   new\_faces (list of lists of tuples): Updated face data referencing the new vertices.
        
*   **How it works:**
    
    1.  **Determine Grid:** It calculates the bounding box of the model and creates a 3D grid based on the grid\_resolution.
        
    2.  **Assign Vertices to Cells:** It iterates through the original vertices and assigns each vertex to a specific cell in the grid based on its coordinates.
        
    3.  **Calculate Cell Representatives:** For each cell containing vertices:
        
        *   It calculates the **barycenter** (average position) of the vertices in that cell. This becomes a new vertex.
            
        *   It calculates the **average texture coordinates** and **average normal vectors** for the vertices in the cell (if texture and normal data are available).
            
    4.  **Update Faces:** It iterates through the original faces and updates the vertex indices to refer to the new barycenter vertices. If all vertices of a face fall into the same cell (or their corresponding barycenters are the same), the face is kept. Degenerate faces (where all vertices collapse to the same point) are avoided.
        
    5.  **Filter and Re-index:** It filters out None values from new\_textures and new\_normals (for cells without original texture or normal data). Then, it re-indexes the texture and normal indices in the new\_faces to match the filtered lists.

In [89]:
def vertex_clustering(vertices, textures, normals, faces, grid_resolution):
    if isinstance(grid_resolution, int):
        grid_resolution = [grid_resolution] * 3

    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    grid_size = (max_coords - min_coords) / np.array(grid_resolution)

    # 2. Assigner les sommets aux cellules
    cell_map = defaultdict(list)
    for i, vertex in enumerate(vertices):
        cell_index = tuple(((vertex - min_coords) // grid_size).astype(int))
        cell_map[cell_index].append(i)

    # 3. Calculer le barycentre, la texture moyenne et la normale moyenne pour chaque cellule
    new_vertices = []
    new_textures = []
    new_normals = []
    cell_barycenter_map = {}  # Associe l'index de la cellule à l'index du nouveau sommet
    for cell_index, vertex_indices in cell_map.items():
        cell_vertices = [vertices[i] for i in vertex_indices]
        if cell_vertices:
            barycenter = np.mean(cell_vertices, axis=0).tolist()
            new_vertices.append(barycenter)
            cell_barycenter_map[cell_index] = len(new_vertices) - 1

            # Calculer la texture moyenne
            cell_textures = [textures[i] for i in vertex_indices if textures]
            if cell_textures:
                avg_texture = np.mean(cell_textures, axis=0).tolist()
                new_textures.append(avg_texture)
            else:
                new_textures.append(None)  # Gérer le cas sans textures

            # Calculer la normale moyenne
            cell_normals = [normals[i] for i in vertex_indices if normals]
            if cell_normals:
                avg_normal = np.mean(cell_normals, axis=0).tolist()
                new_normals.append(avg_normal)
            else:
                new_normals.append(None)  # Gérer le cas sans normales

    # 4. Mettre à jour les faces
    new_faces = []
    for face in faces:
        new_face_data = []
        valid_face = True
        for vertex_index, texture_index, normal_index in face:
            vertex = vertices[vertex_index]
            cell_index = tuple(((vertex - min_coords) // grid_size).astype(int))
            if cell_index in cell_barycenter_map:
                new_vertex_index = cell_barycenter_map[cell_index]

                # Trouver l'index de texture et de normale correspondant au barycentre de la cellule
                new_texture_index = new_vertex_index if new_textures[new_vertex_index] is not None else None
                new_normal_index = new_vertex_index if new_normals[new_vertex_index] is not None else None

                new_face_data.append((new_vertex_index, new_texture_index, new_normal_index))
            else:
                valid_face = False # Skip faces where original vertices are not assigned to any cell
                break

        if valid_face:
            # Éviter les faces dégénérées
            unique_vertices_in_face = list(set(idx for idx, _, _ in new_face_data))
            if len(unique_vertices_in_face) >= 3:
                new_faces.append(new_face_data)

    # Filter out None values from new_textures and new_normals
    new_textures_filtered = [t for t in new_textures if t is not None]
    new_normals_filtered = [n for n in new_normals if n is not None]

    # Re-index faces to match the filtered texture and normal lists
    if new_textures_filtered or new_normals_filtered:
        new_faces_reindexed = []
        for face in new_faces:
            new_face_data_reindexed = []
            for v_idx, vt_idx, vn_idx in face:
                original_vt_index = new_textures.index(new_textures_filtered[vt_idx]) if vt_idx is not None and new_textures_filtered else None
                original_vn_index = new_normals.index(new_normals_filtered[vn_idx]) if vn_idx is not None and new_normals_filtered else None
                new_face_data_reindexed.append((v_idx, original_vt_index, original_vn_index))
            new_faces_reindexed.append(new_face_data_reindexed)
        return new_vertices, new_textures_filtered, new_normals_filtered, new_faces_reindexed
    else:
        return new_vertices, [], [], new_faces

## **3. get\_edges\_from\_faces() Function:**

*   **Purpose:** Extracts unique edges from the list of faces. An edge is defined by a pair of connected vertices.
    
*   **Input:** faces (list of lists of tuples): Face data.
    
*   **Output:** edges (set of tuples): A set containing tuples representing each unique edge. Each tuple contains the sorted indices of the two vertices forming the edge (ensuring that the order doesn't matter, so (1, 2) and (2, 1) represent the same edge).
    
*   **How it works:**
    
    *   It iterates through each face.
        
    *   For each face (assuming it's a triangle), it extracts the vertex indices of the three edges forming the triangle.
        
    *   It sorts the vertex indices of each edge to ensure consistency.
        
    *   It adds the sorted tuple representing the edge to a set, which automatically handles duplicates.
        

## **4. calculate\_edge\_cost() Function:**

*   **Purpose:** Calculates a cost for collapsing a specific edge. In this case, the cost is simply the geometric distance between the two vertices forming the edge.
    
*   **Inputs:**
    
    *   vertices (list of lists of floats): Vertex coordinates.
        
    *   edge (tuple of ints): A tuple containing the indices of the two vertices forming the edge.
        
*   **Output:** float: The Euclidean distance between the two vertices.
    
*   **How it works:**
    
    *   It retrieves the coordinates of the two vertices forming the edge.
        
    *   It calculates the Euclidean distance between these two points using the np.linalg.norm() function.
        

## **5. initialize\_edge\_costs() Function:**

*   **Purpose:** Calculates the cost for each edge in a set of edges and stores them in a min-heap data structure. A min-heap allows efficient retrieval of the edge with the lowest cost.
    
*   **Inputs:**
    
    *   vertices (list of lists of floats): Vertex coordinates.
        
    *   edges (set of tuples): A set of edges.
        
*   **Output:** edge\_heap (list): A min-heap (represented as a list) where each element is a tuple (cost, edge). The heap is ordered based on the cost.
    
*   **How it works:**
    
    *   It iterates through each edge in the edges set.
        
    *   For each edge, it calls calculate\_edge\_cost to get its cost.
        
    *   It pushes the (cost, edge) tuple onto the edge\_heap using heapq.heappush(), which maintains the min-heap property.
        

## **6. collapse\_edge() Function:**

*   **Purpose:** This function performs the core operation of edge collapse, merging the two vertices of a selected edge into a single vertex.
    
*   **Inputs:**
    
    *   vertices (list of lists of floats): Current vertex coordinates.
        
    *   textures (list of lists of floats): Current texture coordinates.
        
    *   normals (list of lists of floats): Current normal vectors.
        
    *   faces (list of lists of tuples): Current face data.
        
    *   edge\_to\_collapse (tuple of ints): The indices of the two vertices forming the edge to be collapsed.
        
    *   collapse\_to (str, optional): Specifies where the new merged vertex should be located. Options are:
        
        *   'midpoint' (default): The new vertex is placed at the midpoint between the two original vertices.
            
        *   'v1': The new vertex takes the position of the first vertex in the edge.
            
        *   'v2': The new vertex takes the position of the second vertex in the edge.
            
*   **Output:** A tuple containing the updated lists of vertices, textures, normals, and faces.
    
*   **How it works:**
    
    1.  **Identify Vertices:** It identifies the indices of the two vertices involved in the edge collapse.
        
    2.  **Calculate New Vertex Position:** Based on the collapse\_to method, it calculates the position of the new merged vertex.
        
    3.  **Update Vertex List:** The position of one of the vertices in the vertices list is updated to the new calculated position.
        
    4.  **Update Faces:** It iterates through the faces and:
        
        *   **Replaces occurrences:** Wherever the indices of the collapsed vertices appear in a face, they are replaced with the index of the vertex that remains (or the index of the new midpoint vertex).
            
        *   **Handles Degeneracy:** Faces that become degenerate (all vertices collapse to the same point) are removed.
            
        *   **Remaps Indices:** After removing one of the original vertices, the indices of subsequent vertices in the faces list are adjusted to reflect the removal.

In [90]:
def get_edges_from_faces(faces):
    """
    Extracts unique edges from a list of faces.

    Args:
        faces (list): List of faces, where each face is a list of vertex indices.

    Returns:
        set: A set of tuples, where each tuple represents an edge (sorted vertex indices).
    """
    edges = set()
    for face in faces:
        v_indices = sorted([face[i][0] for i in range(len(face))])
        for i in range(len(v_indices)):
            for j in range(i + 1, len(v_indices)):
                edges.add(tuple(sorted((v_indices[i], v_indices[j]))))
    return edges

def calculate_edge_cost(vertices, edge):
    """
    Calculates the geometric distance between the vertices of an edge.

    Args:
        vertices (list): List of vertex coordinates.
        edge (tuple): Tuple representing an edge (vertex indices).

    Returns:
        float: The geometric distance between the vertices.
    """
    v1_index, v2_index = edge
    v1_coords = np.array(vertices[v1_index])
    v2_coords = np.array(vertices[v2_index])
    return np.linalg.norm(v1_coords - v2_coords)

def initialize_edge_costs(vertices, edges):
    """
    Calculates the cost for each edge and stores them in a min-heap.

    Args:
        vertices (list): List of vertex coordinates.
        edges (set): Set of edges.

    Returns:
        list: A min-heap of (cost, edge) tuples.
    """
    edge_heap = []
    for edge in edges:
        cost = calculate_edge_cost(vertices, edge)
        heapq.heappush(edge_heap, (cost, edge))
    return edge_heap

def collapse_edge(vertices, textures, normals, faces, edge_to_collapse, collapse_to='midpoint'):
    """
    Collapses a given edge, merging its two vertices.

    Args:
        vertices (list): List of vertex coordinates.
        textures (list): List of texture coordinates.
        normals (list): List of normal vectors.
        faces (list): List of faces.
        edge_to_collapse (tuple): The edge to collapse (vertex indices).
        collapse_to (str): How to calculate the new vertex position ('midpoint', 'v1', 'v2').

    Returns:
        tuple: Updated vertices, textures, normals, and faces.
    """
    v1_index, v2_index = sorted(edge_to_collapse)
    v1 = np.array(vertices[v1_index])
    v2 = np.array(vertices[v2_index])

    # Calculate the new vertex position
    if collapse_to == 'midpoint':
        new_vertex_pos = ((v1 + v2) / 2).tolist()
    elif collapse_to == 'v1':
        new_vertex_pos = v1.tolist()
    elif collapse_to == 'v2':
        new_vertex_pos = v2.tolist()
    else:
        raise ValueError("Invalid collapse_to method.")

    # Add the new vertex
    new_vertex_index = len(vertices)
    new_vertices = vertices
    if v1_index < len(new_vertices):
        new_vertices[v1_index] = new_vertex_pos
    else:
        new_vertices.append(new_vertex_pos)

    new_faces = []
    for face in faces:
        new_face = []
        replace_v1 = False
        replace_v2 = False
        for v_idx, vt_idx, vn_idx in face:
            if v_idx == v1_index:
                replace_v1 = True
                new_face.append((v2_index, vt_idx, vn_idx))
            elif v_idx == v2_index:
                replace_v2 = True
                new_face.append((v2_index, vt_idx, vn_idx))
            else:
                new_face.append((v_idx if v_idx < v1_index else v_idx -1 if v_idx > v1_index else v_idx, vt_idx, vn_idx))

        unique_vertices = set(v_idx for v_idx, _, _ in new_face)
        if len(unique_vertices) >= 3:
            new_faces.append(new_face)

    # Remove the collapsed vertex. Adjust indices in faces.
    # new_vertices = [v for i, v in enumerate(vertices) if i != v1_index]
    # new_faces = []
    # for face in faces:
    #     if all(idx[0] != v1_index for idx in face):
    #         remapped_face = []
    #         for v_idx, vt_idx, vn_idx in face:
    #             new_v_idx = v_idx if v_idx < v1_index else v_idx - 1
    #             remapped_face.append((new_v_idx, vt_idx, vn_idx))
    #         new_faces.append(remapped_face)

    return new_vertices, textures, normals, new_faces

## **7. simplify\_mesh() Function:**

*   **Purpose:** This function implements the iterative edge collapse simplification algorithm. It repeatedly collapses the "cheapest" edge (the one with the smallest cost) until a target number of edge collapses is reached.
    
*   **Inputs:**
    
    *   vertices (list of lists of floats): Original vertex coordinates.
        
    *   textures (list of lists of floats): Original texture coordinates.
        
    *   normals (list of lists of floats): Original normal vectors.
        
    *   faces (list of lists of tuples): Original face data.
        
    *   target\_reduction (int): The desired number of edge collapses to perform.
        
*   **Output:** A tuple containing the simplified lists of vertices, textures, normals, and faces.
    
*   **How it works:**
    
    1.  **Initialization:** It initializes the current mesh data and calculates the initial set of edges and their costs using a min-heap.
        
    2.  **Iterative Collapse:** It enters a loop that continues until the target\_reduction is reached or there are no more edges to collapse:
        
        *   **Get Cheapest Edge:** It retrieves the edge with the lowest cost from the min-heap.
            
        *   **Validate Edge:** It checks if the vertices of the edge are still valid (haven't been merged in a previous collapse).
            
        *   **Collapse Edge:** It calls the collapse\_edge function to merge the two vertices of the selected edge.
            
        *   **Update Edge Heap:** After collapsing an edge, the mesh topology changes, so it rebuilds the edge heap to reflect the new set of edges and their costs (this is a computationally intensive step and can be optimized).
            
    3.  **Return Simplified Mesh:** Once the loop finishes, it returns the simplified mesh data.

In [91]:
def simplify_mesh(vertices, textures, normals, faces, target_reduction):
    """
    Simplifies a mesh by iteratively collapsing edges.

    Args:
        vertices (list): List of vertex coordinates.
        textures (list): List of texture coordinates.
        normals (list): List of normal vectors.
        faces (list): List of faces.
        target_reduction (int): The target number of edge collapses.

    Returns:
        tuple: Simplified vertices, textures, normals, and faces.
    """
    current_vertices = list(vertices)
    current_textures = list(textures)
    current_normals = list(normals)
    current_faces = list(faces)

    edges = get_edges_from_faces(current_faces)
    edge_heap = initialize_edge_costs(current_vertices, edges)

    collapsed_edges_count = 0
    while collapsed_edges_count < target_reduction and edge_heap:
        cost, edge_to_collapse = heapq.heappop(edge_heap)

        # Check if the edge is still valid (vertices haven't been merged yet)
        if edge_to_collapse[0] >= len(current_vertices) or edge_to_collapse[1] >= len(current_vertices) or edge_to_collapse[0] == edge_to_collapse[1]:
            continue

        current_vertices, current_textures, current_normals, current_faces = collapse_edge(
            current_vertices, current_textures, current_normals, current_faces, edge_to_collapse
        )
        collapsed_edges_count += 1

        # Rebuild the edge heap (can be optimized)
        edges = get_edges_from_faces(current_faces)
        edge_heap = initialize_edge_costs(current_vertices, edges)

    return current_vertices, current_textures, current_normals, current_faces

In [92]:
vertices, textures, normals, faces, groups, materials, mtl_file = load_obj("/kaggle/input/objects/obj1.obj")


In [93]:
new_vertices_vc, new_textures_vc, new_normals_vc, new_faces_vc = vertex_clustering(
    vertices, textures, normals, faces, (5, 10, 15)
)

print("Nombre de sommets avant le clustering:", len(vertices))
print("Nombre de sommets après le clustering:", len(new_vertices_vc))
print("Nombre de textures avant le clustering:", len(textures))
print("Nombre de textures après le clustering:", len(new_textures_vc))
print("Nombre de normales avant le clustering:", len(normals))
print("Nombre de normales après le clustering:", len(new_normals_vc))
print("Nombre de faces avant le clustering:", len(faces))
print("Nombre de faces après le clustering:", len(new_faces_vc))

Nombre de sommets avant le clustering: 1369
Nombre de sommets après le clustering: 270
Nombre de textures avant le clustering: 0
Nombre de textures après le clustering: 0
Nombre de normales avant le clustering: 1393
Nombre de normales après le clustering: 270
Nombre de faces avant le clustering: 2734
Nombre de faces après le clustering: 634


In [94]:
# Get edges and initialize edge costs
edges = get_edges_from_faces(new_faces_vc)
print(f"Number of edges: {len(edges)}")
edge_heap = initialize_edge_costs(new_vertices_vc, edges)
print(f"Number of edges in heap: {len(edge_heap)}")

Number of edges: 867
Number of edges in heap: 867


In [95]:
# Simplify the mesh
target_reduction = 100  # Define the number of edges to collapse
simplified_vertices, simplified_textures, simplified_normals, simplified_faces = simplify_mesh(
    new_vertices_vc, new_textures_vc, new_normals_vc, new_faces_vc, target_reduction
)

print("Nombre de sommets après simplification:", len(simplified_vertices))
print("Nombre de faces après simplification:", len(simplified_faces))

Nombre de sommets après simplification: 270
Nombre de faces après simplification: 266
